In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

# Capas Pooling

La operación de _pooling_ ayuda a reducir la dimensionalidad espacial de los feature maps, así reduciendo la cantidad de parámetros de la red. Básicamente son convoluciones, donde el stride habitualmente es igual al tamaño del kernel y donde se calcula alguna función sobre todos los píxeles. Lo más usual es el máximo, el mínimo o el promedio. No solo reducen la dimensionalidad (así logrando un entrenamiento más rápido), sino que además generalmente ayudan en la clasificación, ya que permiten mayor eficacia al momento de generalizar.

Una práctica comun para realizar una arquitectura convolucional es intercalar capas ``Conv2d`` con capas ``Pooling`` hasta llegar a capas ``Dense`` (Feed-Forward) que discriminarán las características aprendidas por las capas anteriores.


## Max Pool

### Método Forward

La operación de _Max Pooling_ es similar a la de convolución, pero sin necesidad de realizar una suma ponderada elemento a elemento, ya que en este caso se selecciona el máximo elemento de cada región particular encerrada por el kernel.

La explicación de su cálculo es sencilla a partir de un ejemplo, supongamos que $x \in \mathbb{R}^{(4\times 4)}$ y $F=S=2$, de modo los $y_{ij}$ se definen como:

$$
\begin{array}{cc}
y_{11} = \max ( x_{11},x_{12},x_{21},x_{22} ) & y_{12} = \max ( x_{13},x_{14},x_{23},x_{24} ) \\
y_{21} = \max ( x_{31},x_{32},x_{41},x_{42} ) & y_{22} = \max ( x_{33},x_{34},x_{43},x_{44} )
\end{array}
$$

Esto nos permite ver que el shape de salida de una capa Max Pool 2D está descripto por la misma ecuación que para el shape de salida de una capa convolucional.

Implementa el método `forward` del modelo `MaxPool2d` en el archivo `edunn/models/pooling.py`.

In [3]:
np.random.seed(123)

x = np.random.rand(2,3,7,7)

kernel_size,stride=2,2

layer=nn.MaxPool2d(kernel_size=kernel_size, stride=stride)

y=layer.forward(x)
y, y.shape

(array([[[[0.69646919, 0.55131477, 0.72904971],
          [0.84943179, 0.72244338, 0.53182759],
          [0.63097612, 0.94416002, 0.62395295]],
 
         [[0.68130077, 0.87545684, 0.66931378],
          [0.88532683, 0.76368284, 0.59443188],
          [0.92513249, 0.69552953, 0.6919703 ]],
 
         [[0.84650623, 0.85445249, 0.38483781],
          [0.98834542, 0.55237008, 0.90191137],
          [0.98352161, 0.80696868, 0.73107304]]],
 
 
        [[[0.7086974 , 0.88785679, 0.78099794],
          [0.7650961 , 0.75164399, 0.85938908],
          [0.81164435, 0.80793821, 0.56221838]],
 
         [[0.99608633, 0.57377411, 0.69983407],
          [0.845533  , 0.78818717, 0.98903451],
          [0.91839747, 0.89497829, 0.31366895]],
 
         [[0.96300447, 0.96749431, 0.86545985],
          [0.5020711 , 0.99303326, 0.37429218],
          [0.88759346, 0.77716246, 0.71099869]]]]),
 (2, 3, 3, 3))

### Método Backward

Conociendo el gradiente de la función de error con respecto a los píxeles de salida, debemos calcular el gradiente total del error con respecto a los píxeles de entrada. Utilizando la regla de la cadena y derivadas parciales podemos llegar a la expresión:

$$
\frac{\partial E}{\partial x_{mn}}=\sum_{ij}\frac{\partial E}{\partial y_{ij}}\frac{\partial y_{ij}}{\partial x_{mn}} 
$$

Tener en cuenta que las capas pooling no tienen parámetros, por lo que no hay que realizar el cálculo del gradiente del kernel.


#### `δE/δx`

El cálculo de los gradientes del error $E$ con respecto al feature map de entrada $x$ se puede hacer derivando parcialmente como se explicó en guías anteriores.

Continuando con el ejemplo previo, es evidente que $\frac{\partial y_{ij}}{\partial x_{mn}}$ es distinto de cero solo si $x_{mn}$ es el máximo elemento de la región de $y_{ij}$, de este modo sucedería que si $x_{12}$ es el máximo elemento de la región que abarca $y_{11}$, entonces:

$$\frac{\partial y_{11}}{\partial x_{12}}=\frac{\partial x_{12}}{\partial x_{12}}=1$$

Y el resto de derivadas con respecto a los otros $x_{mn}$ de la primera región serán cero. La derivada de $E$ con respecto a $x_{12}$ se formula multiplicando el gradiente local por los gradientes provenientes de la siguiente capa, quedando únicamente $\frac{\partial E}{\partial x_{12}}=\frac{\partial E}{\partial y_{11}} \cdot 1 \neq 0$ para la región de $y_{11}$.

* El mismo razonamiento es válido para todas las  $(i,j)$  regiones.

![maxpool_backward.png](img/maxpool_backward.png)

In [4]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[[[0.14877766, 0.        , 0.        , 0.94002901, 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        , 0.8327162 ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.        , 0.        ,
           0.5964869 , 0.        ],
          [0.84605484, 0.        , 0.        , 0.12392301, 0.        ,
           0.        , 0.        ],
          [0.        , 0.01639248, 0.        , 0.        , 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , 0.72118437, 0.        ,
           0.00773751, 0.        ],
          [0.        , 0.        , 0.        , 0.        , 0.        ,
           0.        , 0.        ]],
 
         [[0.        , 0.        , 0.        , 0.        , 0.        ,
           0.        , 0.        ],
          [0.        , 0.08482228, 0.22549841, 0.        , 0.87512453,
           0.        , 0.        ],
          [0.        , 0.        ,

### Comprobaciones con PyTorch

In [5]:
import torch
import torch.nn as tnn

In [6]:
x = torch.from_numpy(x).to(torch.double)

pool = tnn.MaxPool2d(kernel_size=kernel_size, stride=stride, return_indices=True)

x.requires_grad = True
y_torch, indices = pool(x)

In [7]:
utils.check_same(y_torch.detach().numpy(),y)

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [8]:
g = torch.from_numpy(g).to(torch.double)

# Propagar el gradiente hacia atrás a través de la capa de Max Pooling
y_torch.backward(g)

# Imprimir el gradiente de la imagen de entrada
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)

Gradiente de la entrada (δE/δx):
tensor([[[[0.1488, 0.0000, 0.0000, 0.9400, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.8327, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5965, 0.0000],
          [0.8461, 0.0000, 0.0000, 0.1239, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0164, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.7212, 0.0000, 0.0077, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0848, 0.2255, 0.0000, 0.8751, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.5400, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.3636, 0.0000, 0.0000, 0.0000, 0.5681, 0.0000],
          [0.0000, 0.0000, 0.5721, 0.0000, 0.6610, 0.0000, 0.0000],
          [0.2255, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[

In [9]:
utils.check_same(x.grad.numpy(),layer_grad[0])

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [10]:
samples = 100
batch_size=2
din=3 # dimensión de entrada
dout=5 # dimensión de salida
input_shape=(batch_size,din,11,11)

# Verificar las derivadas de un modelo de Flatten
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.MaxPool2d(kernel_size=3)


utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    

MaxPool2d_1 layer:
 ERROR 
δEδx
 Relative error (max):1.00000 (tolerance: 1e-05)
######################## Details: ######################## 
 Input x:
[[[[-8.07699340e-01 -1.27607735e+00  5.53626395e-01  5.53874470e-01
    -6.91200445e-01  3.75582306e-01  2.61272553e-01 -1.28451754e-01
     2.15817020e+00 -8.40878617e-01  1.43050907e-02]
   [-3.82387029e-01 -3.71780015e-01  1.59412004e-01 -2.94395700e-01
    -8.60426760e-01  1.24227498e-01  1.18233165e+00  9.42766380e-01
     2.03044488e-01 -7.35396814e-01  1.86429600e-01]
   [ 1.08464302e+00  1.19118926e+00  3.59687060e-01 -3.64357200e-01
    -2.02752749e-01  7.72045927e-01  6.86346215e-01 -1.75769961e+00
     6.58617565e-01  7.11288340e-01 -8.87191425e-01]
   [-7.64981116e-01 -7.57164098e-01 -6.80262803e-01 -1.41674959e+00
     3.13091930e-01 -7.85719399e-01 -7.03838361e-02 -4.97568783e-01
     2.55177521e-01 -1.01061704e+00  2.45265375e-01]
   [ 3.89781016e-01  8.27594585e-01  1.96776909e+00 -2.09210177e+00
     3.20314334e-01 -7.09

## Avg Pool

### Método Forward

La operación de _Average Pooling_ es similar a la de convolución, donde se calcula el promedio de los elementos que conforman a la región particular encerrada por el kernel.

La explicación de su cálculo es sencilla a partir de un ejemplo, supongamos que $x \in \mathbb{R}^{(4\times 4)}$, $F=2$ y $S=1$, de modo los $y_{ij}$ se definen como:

$$
\begin{array}{ccc}
y_{11} = \text{avg} ( x_{11},x_{12},x_{21},x_{22} ) & 
y_{12} = \text{avg} ( x_{12},x_{13},x_{22},x_{23} ) & 
y_{13} = \text{avg} ( x_{13},x_{14},x_{23},x_{24} ) \\

y_{21} = \text{avg} ( x_{21},x_{22},x_{31},x_{32} ) & 
y_{22} = \text{avg} ( x_{22},x_{23},x_{32},x_{33} ) & 
y_{23} = \text{avg} ( x_{23},x_{24},x_{33},x_{34} ) \\

y_{31} = \text{avg} ( x_{31},x_{32},x_{41},x_{42} ) & 
y_{32} = \text{avg} ( x_{32},x_{33},x_{42},x_{43} ) & 
y_{33} = \text{avg} ( x_{33},x_{34},x_{43},x_{44} )
\end{array}
$$

Esto nos permite ver que el shape de salida de una capa Avg Pool 2D está descripto por la misma ecuación que para el shape de salida de una capa convolucional.

Implementa el método `forward` del modelo `AvgPool2d` en el archivo `edunn/models/pooling.py`.

> TIP: puedes reutilizar la función de forward de la capa `MaxPool2d` y cambiar `np.max` por `np.average`.

In [11]:
np.random.seed(123)

x = np.random.rand(2,3,7,7)
# x = np.random.rand(1,1,4,4)

kernel_size,stride=2,1

layer=nn.AvgPool2d(kernel_size=kernel_size, stride=stride)

y=layer.forward(x)
y, y.shape

(array([[[[0.53709254, 0.34651005, 0.37836544, 0.58575287, 0.57754935,
           0.4755302 ],
          [0.57545033, 0.44838414, 0.27330976, 0.44480771, 0.55775023,
           0.41611967],
          [0.6774817 , 0.56399149, 0.42285259, 0.43810136, 0.43703163,
           0.43907011],
          [0.62464432, 0.51463997, 0.46481825, 0.47749156, 0.40232386,
           0.37739182],
          [0.41582568, 0.51070538, 0.59083882, 0.57764016, 0.51258437,
           0.41477168],
          [0.36768106, 0.65021896, 0.73857843, 0.54487158, 0.64859592,
           0.56115406]],
 
         [[0.45796154, 0.62593177, 0.70382003, 0.58196606, 0.53577054,
           0.46443117],
          [0.65383862, 0.62057376, 0.55818925, 0.54677133, 0.42328492,
           0.49437999],
          [0.62451771, 0.60952831, 0.54938579, 0.4367237 , 0.26211261,
           0.47947425],
          [0.32326738, 0.59029396, 0.59124032, 0.43757057, 0.46422866,
           0.52363772],
          [0.51970816, 0.5119169 , 0.35382125, 

### Método Backward

#### `δE/δx`

El cálculo de los gradientes del error $E$ con respecto al feature map de entrada $x$ se puede hacer derivando parcialmente como se explicó en guías anteriores.

<!-- Al tener que realizar el cálculo de la derivada de una matriz con respecto a otra, se considera cada elemento de la matriz como una variable independiente. Por lo tanto, la derivada de un elemento de la matriz $y$ con respecto a un elemento de la matriz $x$ considerará todas las posibles rutas por las que ese elemento de $x$ puede influir en el elemento de $y$.

De este modo, $\frac{\partial y}{\partial x_{mn}} = \sum_{ij} \frac{\partial y_{ij}}{\partial x_{mn}}$, por la regla de la cadena para derivadas parciales, que nos indica que la derivada de una función con respecto a una variable es la suma de las derivadas de esa función a través de todas las rutas posibles (pudiendo contener términos que sean cero debido a la no dependencia de determinados $y_{ij}$ con $x_{mn}$). -->

Continuando con el ejemplo previo, es evidente que $\frac{\partial y_{ij}}{\partial x_{mn}}$ es distinto de cero solo si $x_{mn}$ pertenece a la región de $y_{ij}$, donde además será igual a $\frac{1}{F_H \cdot F_W}$, entonces para calcular por ejemplo $\frac{\partial y}{\partial x_{12}}$:

$$\frac{\partial y}{\partial x_{12}} = \frac{\partial y_{11}}{\partial x_{12}} + \frac{\partial y_{12}}{\partial x_{12}} + \frac{\partial y_{21}}{\partial x_{12}} + \frac{\partial y_{22}}{\partial x_{12}} = \frac{1}{4} + \frac{1}{4} + 0 + 0$$

ya que:

$$\begin{aligned}
y_{11} &= \frac { x_{11}+\color{red}{x_{12}}\color{default}+x_{21}+x_{22} }{F_H \cdot F_W} \Rightarrow 
    \frac{\partial y_{11}}{\partial x_{12}} = \frac{1}{4} \frac{\partial x_{12}}{\partial x_{12}} = \frac{1}{4} \\
y_{12} &= \frac { \color{red}{x_{12}}\color{default}+x_{13}+x_{22}+x_{23} }{F_H \cdot F_W} \Rightarrow
    \frac{\partial y_{12}}{\partial x_{12}} = \frac{1}{4} \frac{\partial x_{12}}{\partial x_{12}} = \frac{1}{4} \\
y_{21} &= \frac { x_{21}+x_{22}+x_{31}+x_{32} }{F_H \cdot F_W} \Rightarrow 
    \frac{\partial y_{21}}{\partial x_{12}} = 0 \\
y_{22} &= \frac { x_{22}+x_{23}+x_{32}+x_{33} }{F_H \cdot F_W} \Rightarrow 
    \frac{\partial y_{22}}{\partial x_{12}} = 0
\end{aligned}$$


De este modo, el resto de derivadas con respecto a los otros $x_{mn}$ de la primera región serán **en un comienzo** iguales a $\frac{1}{F_H \cdot F_W}$ y según su influencia en la salida debido al valor de $S$ se irán incrementando en esa misma cantidad. 

Esto último mencionado se puede visualizar mejor si $S=2$, donde así es evidente que para la primera región, el resultado parcial de $\frac{\partial y}{\partial x_{mn}}$ para $m,n \in \{1,2\}$ ante la no dependencia entre regiones dadas por el kernel es:

$$
\begin{bmatrix}
1/4 & 1/4 & 0 & 0 \\
1/4 & 1/4 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

* El mismo razonamiento es válido para todas las  $(i,j)$  regiones.


In [12]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})

In [13]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)
# g = np.ones_like(y)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[[[0.0372, 0.2722, 0.4432, 0.4197, 0.2425, 0.1801, 0.1491],
          [0.0413, 0.4566, 0.6254, 0.4428, 0.3201, 0.4553, 0.3679],
          [0.0950, 0.4103, 0.4592, 0.2215, 0.2770, 0.5834, 0.3840],
          [0.1655, 0.4051, 0.4949, 0.5448, 0.5794, 0.6922, 0.4023],
          [0.2136, 0.4434, 0.3440, 0.4675, 0.7320, 0.6654, 0.2867],
          [0.1520, 0.3789, 0.3208, 0.4285, 0.5730, 0.5181, 0.2796],
          [0.0130, 0.1147, 0.1948, 0.3074, 0.2209, 0.2367, 0.2300]],
 
         [[0.1702, 0.3963, 0.3779, 0.3549, 0.2869, 0.1713, 0.0874],
          [0.2677, 0.6825, 0.6590, 0.5077, 0.5818, 0.6327, 0.3144],
          [0.1847, 0.5320, 0.5081, 0.2729, 0.4306, 0.6273, 0.3088],
          [0.3078, 0.6720, 0.6101, 0.5372, 0.4811, 0.3328, 0.1430],
          [0.2499, 0.5308, 0.4946, 0.4766, 0.5193, 0.4087, 0.1523],
          [0.1705, 0.2936, 0.3286, 0.2825, 0.2972, 0.4967, 0.2765],
          [0.1411, 0.1890, 0.2171, 0.2231, 0.1234, 0.2549, 0.1854]],
 
         [[0.1399, 0.2236, 0.2195, 0.3092,

### Comprobaciones con PyTorch

In [14]:
import torch
import torch.nn as tnn

In [15]:
x = torch.from_numpy(x).to(torch.double)

pool = tnn.AvgPool2d(kernel_size=kernel_size, stride=stride)

x.requires_grad = True
y_torch = pool(x)

In [16]:
utils.check_same(y_torch.detach().numpy(),y)

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [17]:
g = torch.from_numpy(g).to(torch.double)

# Propagar el gradiente hacia atrás a través de la capa de Max Pooling
y_torch.backward(g)

# Imprimir el gradiente de la imagen de entrada
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)

Gradiente de la entrada (δE/δx):
tensor([[[[0.0372, 0.2722, 0.4432, 0.4197, 0.2425, 0.1801, 0.1491],
          [0.0413, 0.4566, 0.6254, 0.4428, 0.3201, 0.4553, 0.3679],
          [0.0950, 0.4103, 0.4592, 0.2215, 0.2770, 0.5834, 0.3840],
          [0.1655, 0.4051, 0.4949, 0.5448, 0.5794, 0.6922, 0.4023],
          [0.2136, 0.4434, 0.3440, 0.4675, 0.7320, 0.6654, 0.2867],
          [0.1520, 0.3789, 0.3208, 0.4285, 0.5730, 0.5181, 0.2796],
          [0.0130, 0.1147, 0.1948, 0.3074, 0.2209, 0.2367, 0.2300]],

         [[0.1702, 0.3963, 0.3779, 0.3549, 0.2869, 0.1713, 0.0874],
          [0.2677, 0.6825, 0.6590, 0.5077, 0.5818, 0.6327, 0.3144],
          [0.1847, 0.5320, 0.5081, 0.2729, 0.4306, 0.6273, 0.3088],
          [0.3078, 0.6720, 0.6101, 0.5372, 0.4811, 0.3328, 0.1430],
          [0.2499, 0.5308, 0.4946, 0.4766, 0.5193, 0.4087, 0.1523],
          [0.1705, 0.2936, 0.3286, 0.2825, 0.2972, 0.4967, 0.2765],
          [0.1411, 0.1890, 0.2171, 0.2231, 0.1234, 0.2549, 0.1854]],

         [[

In [18]:
utils.check_same(x.grad.numpy(),layer_grad[0])

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [19]:
samples = 100
batch_size=2
din=3 # dimensión de entrada
dout=5 # dimensión de salida
input_shape=(batch_size,din,11,11)

# Verificar las derivadas de un modelo de Flatten
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.AvgPool2d(kernel_size=3, stride=3)


utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    

AvgPool2d_1 layer:
SUCCESS 72600 partial derivatives checked (100 random input samples)
